In [169]:
import pandas as pd
import datetime as dt
import time
import requests
import numpy as np

import joblib

import warnings
warnings.filterwarnings('ignore')

In [2]:
schedule = requests.get('https://statsapi.web.nhl.com/api/v1/schedule')

In [3]:
schedule = schedule.json()

In [13]:
game_link = schedule['dates'][0]['games'][0]['link']

In [18]:
away_team = schedule['dates'][0]['games'][0]['teams']['away']['team']['name']

In [19]:
home_team = schedule['dates'][0]['games'][0]['teams']['home']['team']['name']

In [90]:
base_url = 'https://statsapi.web.nhl.com'

In [91]:
game_link

'/api/v1/game/2020030236/feed/live'

In [92]:
game_pull = requests.get(f'{base_url}{game_link}')

In [61]:
events_test =requests.get('https://statsapi.web.nhl.com/api/v1/game/2020020001/feed/live')
events_test = events_test.json()

In [154]:
test_link = '/api/v1/game/2020020001/feed/live'

In [97]:
game_pull = game_pull.json()

In [41]:
modeling_df = pd.DataFrame()
game_events = pd.DataFrame()

In [83]:
def update_game_events(current_pull):
    gameplay = current_pull['liveData']['plays']['allPlays']
    output_df = pd.DataFrame()
    for n in range(0,len(gameplay),1):
        event = gameplay[n]['result']['event']
        period = gameplay[n]['about']['period']
        periodTime = gameplay[n]['about']['periodTime']
        try:
            team = gameplay[n]['team']['name']
        except:
            team = np.nan
        home_team = current_pull['gameData']['teams']['home']['name']
        try:
            x_cords = gameplay[n]['coordinates']['x']
        except:
            x_cords = np.nan
        try:
            y_cords = gameplay[n]['coordinates']['y']
        except:
            y_cords = np.nan
        temp_dic = {'event': event, 'team': team,'x_coordinate':x_cords, 'y_coordinate': y_cords,
                       'period':period, 'period_time': periodTime, 'home_team': home_team}
        temp_df = pd.DataFrame(temp_dic, index=[n])
        output_df = pd.concat([output_df, temp_df], axis=0, ignore_index = True)
    return output_df


In [86]:
update_test = update_game_events(events_test).dropna()

In [46]:
def ice_tilt(game_df):
    game_ice_tilt = game_df[(game_df['event'] == 'Faceoff') | (game_df['event'] == 'Shot') | (game_df['event'] == 'Hit') | (game_df['event'] == 'Missed Shot')]                
    mask = (game_ice_tilt['period']==2)
    game_ice_tilt['x_coordinate'][mask] = game_ice_tilt['x_coordinate'] * -1
    tilt = game_ice_tilt['x_coordinate'].sum()
    return tilt

In [87]:
ice_tilt(update_test)

<ipython-input-46-a9b4cecefa03>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_ice_tilt['x_coordinate'][mask] = game_ice_tilt['x_coordinate'] * -1
C:\Users\DanAn\anaconda3secondinstall\lib\site-packages\pandas\core\series.py:1020: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


1058.0

In [127]:
def update_box_scores(current_pull):
    box_pull = current_pull['liveData']['boxscore']
    output_df = pd.DataFrame()
    for location in ['away', 'home']:
        shots = box_pull['teams'][location]['teamStats']['teamSkaterStats']['shots']
        blocked_shots = box_pull['teams'][location]['teamStats']['teamSkaterStats']['blocked']
        pim = box_pull['teams'][location]['teamStats']['teamSkaterStats']['pim']
        pp_percent = box_pull['teams'][location]['teamStats']['teamSkaterStats']['powerPlayPercentage']
        pp_goals = box_pull['teams'][location]['teamStats']['teamSkaterStats']['powerPlayGoals']
        pp_chances = box_pull['teams'][location]['teamStats']['teamSkaterStats']['powerPlayOpportunities']
        faceoff_percent = box_pull['teams'][location]['teamStats']['teamSkaterStats']['faceOffWinPercentage']
        takeaways = box_pull['teams'][location]['teamStats']['teamSkaterStats']['takeaways']
        giveaways = box_pull['teams'][location]['teamStats']['teamSkaterStats']['giveaways']
        hits = box_pull['teams'][location]['teamStats']['teamSkaterStats']['hits']
        home_status = location
        temp_dic = {
            'shots': shots, 'blocked_shots': blocked_shots,
            'penalty_minutes': pim, 'power_play_percentage': pp_percent, 'power_play_goals': pp_goals,
            'power_play_chances': pp_chances, 'faceoff_percent':faceoff_percent,'takeaways':takeaways,
            'giveaways' : giveaways,'hits':hits,'home_status': home_status
            }
        temp_df = pd.DataFrame(temp_dic, index=[0])
        output_df = pd.concat([output_df, temp_df], axis=0, ignore_index=True)
    return output_df

In [146]:
update_box_scores(events_test)

,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits,home_status
0,34,11,6,33.3,1.0,3.0,42.0,8,10,23,away
1,27,13,6,66.7,2.0,3.0,58.0,6,10,31,home


In [176]:
def update_differences_df(df):
    df['power_play_percentage'] = pd.to_numeric( df['power_play_percentage'])
    df['faceoff_percent'] = pd.to_numeric( df['faceoff_percent'])
    diffs = df.iloc[1][0:10] - df.iloc[0][0:10]
    output_df = pd.DataFrame(diffs.to_dict(), index=[0])
    return output_df
    

In [177]:
update_differences_df(diff_test)

,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits
0,-7,2,0,33.4,1.0,0.0,16.0,-2,0,8


In [178]:
diff_test = box_scores_update(events_test)

In [179]:
    modeling_df = pd.DataFrame()
    game_events = pd.DataFrame()
    differences_df = pd.DataFrame()

In [226]:
def game_scrape(game_link):
    base_url = 'https://statsapi.web.nhl.com'
    current_pull_raw = requests.get(f'{base_url}{game_link}')
    current_pull = current_pull_raw.json()
    modeling_df = update_box_scores(current_pull)
    differences_df = update_differences_df(modeling_df)
    game_events = update_game_events(current_pull)
    tilt = ice_tilt(game_events)
    differences_df['ice_tilt'] = tilt * -1
    differences_df['time_stamp'] = current_pull['metaData']['timeStamp']
    differences_df['period'] = current_pull['liveData']['plays']['allPlays'][-1]['about']['period']
    differences_df['period_time'] = current_pull['liveData']['plays']['allPlays'][-1]['about']['periodTime']
    modeling_df['time_stamp'] = current_pull['metaData']['timeStamp']
    modeling_df['period'] = current_pull['liveData']['plays']['allPlays'][-1]['about']['period']
    modeling_df['period_time'] = current_pull['liveData']['plays']['allPlays'][-1]['about']['periodTime']
    return differences_df, modeling_df
    
    
    

In [216]:
diffs, boxes = game_scrape(game_link)

In [217]:
diffs

,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits,ice_tilt,time_stamp,period,period_time
0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,-0.0,20210611_004225,1,00:00


In [218]:
boxes

,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits,home_status,time_stamp,period,period_time
0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,away,20210611_004225,1,00:00
1,0,0,0,0.0,0.0,0.0,0.0,0,0,0,home,20210611_004225,1,00:00


In [219]:
diffs, boxes = game_scrape(test_link)

In [220]:
diffs

,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits,ice_tilt,time_stamp,period,period_time
0,-7,2,0,33.4,1.0,0.0,16.0,-2,0,8,-1058.0,20210116_190940,3,20:00


In [221]:
boxes

,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits,home_status,time_stamp,period,period_time
0,34,11,6,33.3,1.0,3.0,42.0,8,10,23,away,20210116_190940,3,20:00
1,27,13,6,66.7,2.0,3.0,58.0,6,10,31,home,20210116_190940,3,20:00


In [222]:
rfc = joblib.load('./models/random_forest_classifier.pkl')
log_reg = joblib.load('./models/log_reg.pkl')

In [199]:
game_pull['metaData']['timeStamp']

'20210610_233606'

In [233]:
test_game_1_models = pd.DataFrame()
test_game_1_diffs = pd.DataFrame()
for i in range(1,20,1):
    differences, models = game_scrape(game_link)
    test_game_1_models = pd.concat([test_game_1_models, models], axis=0, ignore_index=True)
    test_game_1_diffs = pd.concat([test_game_1_diffs, differences], axis=0, ignore_index=True)
    time.sleep(60)

In [238]:
test_game_1_models

,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits,home_status,time_stamp,period,period_time
0,8,2,0,0.0,0.0,0.0,45.5,0,0,5,away,20210611_012712,1,08:10
1,6,3,0,0.0,0.0,0.0,54.5,4,2,9,home,20210611_012712,1,08:10
2,9,2,0,0.0,0.0,0.0,45.5,0,0,9,away,20210611_012807,1,09:16
3,6,5,0,0.0,0.0,0.0,54.5,6,3,12,home,20210611_012807,1,09:16
4,9,2,0,0.0,0.0,0.0,45.5,0,0,9,away,20210611_012914,1,10:11
5,6,5,0,0.0,0.0,0.0,54.5,6,3,12,home,20210611_012914,1,10:11
6,10,2,0,0.0,0.0,0.0,45.5,0,0,9,away,20210611_013009,1,11:14
7,6,5,0,0.0,0.0,0.0,54.5,7,4,13,home,20210611_013009,1,11:14
8,10,2,0,0.0,0.0,0.0,45.5,0,0,9,away,20210611_013104,1,11:36
9,6,7,0,0.0,0.0,0.0,54.5,8,4,17,home,20210611_013104,1,11:36


In [237]:
test_game_1_diffs 

""


In [243]:
X = test_game_1_models.drop(columns=['home_status','time_stamp','period','period_time'])

In [244]:
box_model = joblib.load('./models/box_score_log_reg_2020')

In [246]:
box_model.predict_proba(X)

array([[0.75756193, 0.24243807],
       [0.73222567, 0.26777433],
       [0.77707815, 0.22292185],
       [0.70669478, 0.29330522],
       [0.77707815, 0.22292185],
       [0.70669478, 0.29330522],
       [0.77311855, 0.22688145],
       [0.70399312, 0.29600688],
       [0.77311855, 0.22688145],
       [0.6939951 , 0.3060049 ],
       [0.77885799, 0.22114201],
       [0.6939951 , 0.3060049 ],
       [0.78449271, 0.21550729],
       [0.68693845, 0.31306155],
       [0.77087795, 0.22912205],
       [0.68554708, 0.31445292],
       [0.77087795, 0.22912205],
       [0.68554708, 0.31445292],
       [0.77087795, 0.22912205],
       [0.68554708, 0.31445292],
       [0.77984458, 0.22015542],
       [0.65540996, 0.34459004],
       [0.77984458, 0.22015542],
       [0.6502619 , 0.3497381 ],
       [0.7766577 , 0.2233423 ],
       [0.65444989, 0.34555011],
       [0.77549719, 0.22450281],
       [0.627216  , 0.372784  ],
       [0.76767847, 0.23232153],
       [0.62538036, 0.37461964],
       [0.

In [ ]:
game = request.get('https://statsapi.web.nhl.com/api/v1/game/2020030236/feed/live')
game = game.json()

In [239]:
def get_game_status(game_link):
    base_url = 'https://statsapi.web.nhl.com'
    current_pull_raw = requests.get(f'{base_url}{game_link}')
    current_pull = current_pull_raw.json()
    return current_pull['gameData']['status']['abstractGameState']
    
    

In [240]:
get_game_status(game_link)

'Live'

In [250]:
for i in range(1,31,1):
    differences, models = game_scrape(game_link)
    test_game_1_models = pd.concat([test_game_1_models, models], axis=0, ignore_index=True)
    test_game_1_diffs = pd.concat([test_game_1_diffs, differences], axis=0, ignore_index=True)
    time.sleep(60)

KeyboardInterrupt: 

In [251]:
test_game_1_models

,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits,home_status,time_stamp,period,period_time
0,8,2,0,0.0,0.0,0.0,45.5,0,0,5,away,20210611_012712,1,08:10
1,6,3,0,0.0,0.0,0.0,54.5,4,2,9,home,20210611_012712,1,08:10
2,9,2,0,0.0,0.0,0.0,45.5,0,0,9,away,20210611_012807,1,09:16
3,6,5,0,0.0,0.0,0.0,54.5,6,3,12,home,20210611_012807,1,09:16
4,9,2,0,0.0,0.0,0.0,45.5,0,0,9,away,20210611_012914,1,10:11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,13,23,2,0.0,0.0,0.0,51.6,14,11,32,home,20210611_023525,2,14:24
96,20,7,0,100.0,1.0,1.0,46.9,1,1,27,away,20210611_023623,2,14:27
97,14,23,2,0.0,0.0,0.0,53.1,14,11,32,home,20210611_023623,2,14:27
98,20,7,0,100.0,1.0,1.0,44.1,1,1,27,away,20210611_023723,2,15:05


In [252]:
test_game_1_diffs

,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits,ice_tilt,time_stamp,period,period_time
0,-2,10,0,0.0,0.0,0.0,10.0,11,5,8,-1083.0,20210611_020722,2,00:00
1,-3,10,0,0.0,0.0,0.0,10.0,11,5,8,-1130.0,20210611_020817,2,01:27
2,-3,10,0,0.0,0.0,0.0,10.0,11,5,8,-1038.0,20210611_020923,2,02:18
3,-3,12,0,0.0,0.0,0.0,10.0,11,7,8,-1107.0,20210611_021018,2,02:23
4,-3,13,2,0.0,0.0,0.0,4.8,11,7,9,-1169.0,20210611_021113,2,03:03
5,-4,13,2,0.0,0.0,-1.0,0.0,11,7,9,-1238.0,20210611_021222,2,03:47
6,-4,14,2,-100.0,-1.0,-1.0,4.4,11,7,9,-1202.0,20210611_021317,2,04:03
7,-4,14,2,-100.0,-1.0,-1.0,4.4,11,7,9,-1143.0,20210611_021412,2,04:48
8,-4,14,2,-100.0,-1.0,-1.0,4.4,11,7,10,-1056.0,20210611_021522,2,05:30
9,-6,14,2,-100.0,-1.0,-1.0,8.4,11,7,10,-1222.0,20210611_021624,2,06:40


In [266]:
test_game_1_diffs.to_csv('./live_scrapes/avs_vegas_partial_second_diffs.csv')
test_game_1_models.to_csv('./live_scrapes/avs_vegas_partial_second_box_scores.csv')

In [253]:
rfc.predict(test_game_1_diffs.drop(columns=['time_stamp','time_stamp','period','period_time']))

array(['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no'],
      dtype=object)

In [254]:
rfc.predict_proba(test_game_1_diffs.drop(columns=['time_stamp','time_stamp','period','period_time']))

array([[0.48843059, 0.51156941],
       [0.48843059, 0.51156941],
       [0.48843059, 0.51156941],
       [0.48843059, 0.51156941],
       [0.46933786, 0.53066214],
       [0.47475214, 0.52524786],
       [0.60165073, 0.39834927],
       [0.60721701, 0.39278299],
       [0.60721701, 0.39278299],
       [0.61173347, 0.38826653],
       [0.59902202, 0.40097798],
       [0.59902202, 0.40097798],
       [0.59902202, 0.40097798],
       [0.59902202, 0.40097798],
       [0.58794663, 0.41205337],
       [0.61308796, 0.38691204],
       [0.61308796, 0.38691204],
       [0.61308796, 0.38691204],
       [0.60201257, 0.39798743],
       [0.60201257, 0.39798743],
       [0.60647469, 0.39352531],
       [0.60201257, 0.39798743],
       [0.60201257, 0.39798743],
       [0.60201257, 0.39798743],
       [0.60201257, 0.39798743],
       [0.60201257, 0.39798743],
       [0.60201257, 0.39798743],
       [0.60201257, 0.39798743],
       [0.60201257, 0.39798743],
       [0.60201257, 0.39798743],
       [0.

In [ ]:
test_game_2_models = pd.DataFrame()
test_game_2_diffs = pd.DataFrame()
for i in range(1,51,1):
    differences, models = game_scrape(game_link)
    test_game_2_models = pd.concat([test_game_2_models, models], axis=0, ignore_index=True)
    test_game_2_diffs = pd.concat([test_game_2_diffs, differences], axis=0, ignore_index=True)
    time.sleep(60)

In [261]:
game_status = get_game_status(game_link)
test_game_2_models = pd.DataFrame()
test_game_2_diffs = pd.DataFrame()
while game_status == 'Live':
    differences, models = game_scrape(game_link)
    test_game_2_models = pd.concat([test_game_2_models, models], axis=0, ignore_index=True)
    test_game_2_diffs = pd.concat([test_game_2_diffs, differences], axis=0, ignore_index=True)
    game_status = get_game_status(game_link)
    time.sleep(60)
    

In [262]:
game_status = get_game_status(game_link)

In [260]:
game_status

'Live'

In [263]:
test_game_2_models

,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits,home_status,time_stamp,period,period_time
0,22,7,0,100.0,1.0,1.0,50.0,1,1,31,away,20210611_030529,3,00:22
1,16,24,2,0.0,0.0,0.0,50.0,16,12,37,home,20210611_030529,3,00:22
2,22,7,0,100.0,1.0,1.0,48.8,1,1,31,away,20210611_030627,3,01:21
3,17,25,2,0.0,0.0,0.0,51.2,16,13,37,home,20210611_030627,3,01:21
4,22,7,0,100.0,1.0,1.0,48.8,1,1,31,away,20210611_030727,3,01:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,23,28,2,0.0,0.0,1.0,50.0,20,15,45,home,20210611_033938,3,18:55
70,32,17,2,100.0,1.0,1.0,50.0,4,1,41,away,20210611_034038,3,19:21
71,23,28,2,0.0,0.0,1.0,50.0,20,15,45,home,20210611_034038,3,19:21
72,33,17,2,100.0,1.0,1.0,51.8,4,1,41,away,20210611_034136,3,20:00


In [264]:
test_game_2_diffs

,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits,ice_tilt,time_stamp,period,period_time
0,-6,17,2,-100.0,-1.0,-1.0,0.0,15,11,6,-1343.0,20210611_030529,3,00:22
1,-5,18,2,-100.0,-1.0,-1.0,2.4,15,12,6,-1283.0,20210611_030627,3,01:21
2,-5,18,2,-100.0,-1.0,-1.0,2.4,15,12,6,-1214.0,20210611_030727,3,01:21
3,-5,18,2,-100.0,-1.0,-1.0,4.8,16,12,6,-1168.0,20210611_030825,3,02:58
4,-5,19,2,-100.0,-1.0,-1.0,4.8,16,12,6,-1226.0,20210611_030923,3,03:49
5,-5,19,2,-100.0,-1.0,-1.0,4.8,16,12,4,-1016.0,20210611_031023,3,04:41
6,-6,18,2,-100.0,-1.0,-1.0,4.8,15,12,2,-1263.0,20210611_031132,3,05:28
7,-7,18,2,-100.0,-1.0,-1.0,4.8,15,13,1,-1364.0,20210611_031230,3,05:33
8,-7,18,2,-100.0,-1.0,-1.0,4.6,15,13,1,-1364.0,20210611_031330,3,06:12
9,-10,18,2,-100.0,-1.0,-1.0,4.6,15,13,1,-1644.0,20210611_031428,3,06:37


In [265]:
rfc.predict(test_game_2_diffs.drop(columns=['time_stamp','time_stamp','period','period_time']))

array(['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no'], dtype=object)

In [267]:
test_game_2_diffs.to_csv('./live_scrapes/avs_vegas_third_diffs.csv')
test_game_2_models.to_csv('./live_scrapes/avs_vegas_third_box_scores.csv')

In [ ]:
#Live